In [1]:
import torch, torchvision, numpy as np
from pix2code_resnet18_free import Vocabulary, Pix2codeDataset, collate_fn, EncoderCNN, DecoderRNN

In [2]:
batch_size = 1     # length of test dataset
num_workers = 1
embed_size = 256
hidden_size = 512
num_layers = 1
crop_size = 224
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# %% Paths
data_path = "../data/data_bootstrap/"
#train_path = data_path + "processed_data/data_train/"
test_path = data_path + "processed_data/data_test/"
vocab_path = data_path + "bootstrap.vocab"

In [4]:
with open(vocab_path, 'r') as file:
    content = file.read()
words = content.split(' '); del content
len(words)

vocab = Vocabulary()
for word in words:
    vocab.add_word(word)
vocab.add_word(' ')
vocab.add_word('<unk>') # if we find an unknown word
vocab_size = len(vocab)
print(f"total length of vocab is : {vocab_size}")

total length of vocab is : 19


In [5]:
# %%  Transform to modify images for pre-trained ResNet base and train_dataset
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((crop_size, crop_size)), # Match resnet size
    torchvision.transforms.ToTensor(),
    # See for : http://pytorch.org/docs/master/torchvision/models.html
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
test_dataset = Pix2codeDataset(data_path=test_path, vocab=vocab, transform=transform)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size,
                         shuffle=True, num_workers=num_workers, collate_fn=collate_fn)

Created dataset of   175 items from
../data/data_bootstrap/processed_data/data_test/


## model parameters saved

trained 100 epochs 

In [28]:
encoder = EncoderCNN(embed_size).to(device)
decoder = DecoderRNN(embed_size, hidden_size, len(vocab), num_layers).to(device)

folder_ = "./Logs/2018-08-04-03-40-07/"
epoch_ = 100
encoder.load_state_dict(torch.load(f"{folder_}encoder_epoch{epoch_}.pkl",map_location='cpu'))
decoder.load_state_dict(torch.load(f"{folder_}decoder_epoch{epoch_}.pkl",map_location='cpu'))

In [39]:
for index in target[1:-1].numpy().astype(int):
    print(vocab.idx2word[index], end="")

header{btn-active,btn-inactive,btn-inactive,btn-inactive}row{quadruple{small-title,text,btn-orange}quadruple{small-title,text,btn-green}quadruple{small-title,text,btn-orange}quadruple{small-title,text,btn-orange}}row{double{small-title,text,btn-orange}double{small-title,text,btn-red}}

In [47]:
for j, data in enumerate(test_loader):
    # (images_test, captions_test, lengths_test)
    print(j)
    #features = encoder(images_test)
    #features

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87


In [42]:
for images, captions, lengths in test_loader:
    #print(images.size())
    features = encoder(images)
    #print(features.size())
    break
sampled_ids = []
inputs = features.unsqueeze(1)
inputs.size()
states = None
for i in range(77):
    hiddens, states = decoder.lstm(inputs, states)
    outputs = decoder.fc(hiddens.squeeze(1))
    _, predicted = outputs.max(1)
    sampled_ids.append(predicted)
    inputs = decoder.embed(predicted)
    inputs = inputs.unsqueeze(1)
sampled_ids = torch.stack(sampled_ids, 1)  
sampled_ids.size()

torch.Size([2, 77])

In [44]:
n = 1
print(sampled_ids[n,:])
print(captions[n,:])

tensor([ 12,  13,   1,   7,   0,  14,   0,   7,   2,   6,   1,   5,
          1,   3,   0,   4,   0,   8,   2,   5,   1,   3,   0,   4,
          0,   8,   2,   5,   1,   3,   0,   4,   0,   8,   2,   5,
          1,   3,   0,   4,   0,   8,   2,   2,   6,   1,  11,   1,
          3,   0,   4,   0,   8,   2,  11,   1,   3,   0,   4,   0,
          8,   2,   2,  15,  15,  15,  12,  13,   1,   7,   0,   7,
          0,   7,   0,  14,   0])
tensor([ 12,  13,   1,   7,   0,  14,   0,   7,   2,   6,   1,  11,
          1,   3,   0,   4,   0,   8,   2,  11,   1,   3,   0,   4,
          0,   9,   2,   2,   6,   1,   5,   1,   3,   0,   4,   0,
          8,   2,   5,   1,   3,   0,   4,   0,  10,   2,   5,   1,
          3,   0,   4,   0,   8,   2,   5,   1,   3,   0,   4,   0,
         10,   2,   2,   6,   1,  18,   1,   3,   0,   4,   0,   9,
          2,   2,  15])


## model saved

100 epochs pretrained + further 200 epochs

Adam optimizer

In [9]:
encoder = torch.load("./Logs/20180805-123547/encoder_epoch199.pkl")
decoder = torch.load("./Logs/20180805-123547/decoder_epoch199.pkl")

In [10]:
prediction_list = []
caption_list = []

for images, captions, lengths in test_loader:
    features = encoder(images)
    caption_list.append(captions.numpy()[0,:].tolist())   # there is an extra batch dimension
    
    sampled_ids = []
    inputs = features.unsqueeze(1)
    states = None
    for i in range(100):            # predict 100 words at most
        # predict    
        hiddens, states = decoder.lstm(inputs, states)
        outputs = decoder.fc(hiddens.squeeze(1))
        _, predicted = outputs.max(1)
        
        # store prediction
        sampled_ids.append(predicted.item())
        
        # if predicts <END>, break
        # 把15(<END>)后面的单词全部设为0，因为训练中并没有教模型15以后的时序列应该怎么预测
        if sampled_ids[-1] == vocab.word2idx['<END>']:
            break
        
        # prediction as the next input
        inputs = decoder.embed(predicted)
        inputs = inputs.unsqueeze(1)
        
    prediction_list.append(sampled_ids)

save incorrect prediction

In [12]:
info = ""
for i in range(len(prediction_list)):
    info_ = "[{:4d}/{:4d}] :\n".format( i+1,len(prediction_list) )
    
    count = 0
    if len(prediction_list[i]) > len(caption_list[i]):
        for n in range(len(prediction_list[i])):
            if n >= len(caption_list[i]):
                info_ += "\t\t{:10s} \t-->\t {:10s}\n".format("None", vocab.idx2word[prediction_list[i][n]])
                count += 1
            elif prediction_list[i][n] != caption_list[i][n]:
                info_ += "\t\t{:10s} \t-->\t {:10s}\n".format(vocab.idx2word[caption_list[i][n]], vocab.idx2word[prediction_list[i][n]])
                count += 1
            else:
                pass
    
    if len(prediction_list[i]) < len(caption_list[i]):
        for n in range(len(caption_list[i])):
            if n >= len(prediction_list[i]):
                info_ += "\t\t{:10s} \t-->\t {:10s}\n".format(vocab.idx2word[caption_list[i][n]], "None")
                count += 1
            elif prediction_list[i][n] != caption_list[i][n]:
                info_ += "\t\t{:10s} \t-->\t {:10s}\n".format(vocab.idx2word[caption_list[i][n]], vocab.idx2word[prediction_list[i][n]])
                count += 1
            else:
                pass
    
    if count > 0:
        info += info_

#with open("./compare.txt", "w+") as file:
    #file.write(info)
print(info)

[   7/ 175] :
		btn-green  	-->	 btn-orange
		<unk>      	-->	 double    
		btn-orange 	-->	 btn-green 
		}          	-->	 double    
		<END>      	-->	 {         
		None       	-->	 small-title
		None       	-->	 ,         
		None       	-->	 text      
		None       	-->	 ,         
		None       	-->	 btn-red   
		None       	-->	 }         
		None       	-->	 }         
		None       	-->	 <END>     
[  30/ 175] :
		btn-orange 	-->	 btn-red   
		btn-green  	-->	 btn-orange
		btn-red    	-->	 btn-orange
		btn-red    	-->	 btn-orange
		btn-orange 	-->	 btn-red   
		quadruple  	-->	 <unk>     
		btn-green  	-->	 btn-orange
		quadruple  	-->	 }         
		{          	-->	 <END>     
		small-title 	-->	 None      
		,          	-->	 None      
		text       	-->	 None      
		,          	-->	 None      
		btn-green  	-->	 None      
		}          	-->	 None      
		quadruple  	-->	 None      
		{          	-->	 None      
		small-title 	-->	 None      
		,          	-->	 None      
		text   